### Generate Individual vDAC Components

In [2]:
import hdl21 as h
from vdac import *
import sky130_hdl21 as s
from sitepdks import *

import vlsirtools.spice as vsp

h.elaborate(gen_vDAC())

Module(name=gen_vDAC(npar=2 ncells=6))

### vDAC Fuzzing

We want to cycle through all of the $2^8$ digital signals that can be input and see how Pretl's "ana-ligital" vDAC holds up. 

In [7]:
# def get_sim(width=8,inp=0):

#     @h.sim.sim
#     class vDAC_Fuzzing:

#         @h.module
#         class Tb:
#             VSS = h.Port()
#             out = h.Signal()
            
#             vdc = h.Vdc(dc=1.8)(n=VSS)
#             vdac = gen_vDAC(ncells=width,npar=2)(vout=out, VSS=VSS, VDD=vdc.p)
#             vsources = bus_signal(width=width,inp=inp)(vss=VSS)
            
#             vdac.inp_bus = vsources.vout
#             vdac.enable = vdc.p

#         op = h.sim.Op()
#         inc_base = s.install.include(h.pdk.Corner.TYP)
#         inc_cells = h.sim.Include(
#                 sky130.install.pdk_path /
#                 "libs.ref/sky130_fd_sc_hd/spice/sky130_fd_sc_hd.spice"
#         )

#     return vDAC_Fuzzing

@h.paramclass
class vDAC_Fuzzing_Params:
    width = h.Param(default=8, dtype=int, desc="Width of the DAC")
    inp = h.Param(default=0, dtype=int, desc="Input value")

@h.generator
def gen_vDAC_Fuzzing(params : vDAC_Fuzzing_Params) -> h.Module:
    
    @h.module
    class vDAC:
        VSS = h.Port()
        out = h.Signal()

        vdc = h.Vdc(dc=1.8)(n=VSS)
        vdac = gen_vDAC(ncells=params.width, npar=2)(vout=out, VSS=VSS, VDD=vdc.p)
        vsources = bus_signal(width=params.width, inp=params.inp)(vss=VSS)

        vdac.inp_bus = vsources.vout
        vdac.enable = vdc.p

    return vDAC

def get_sim(width=8, inp=0):

    sim = h.sim.Sim(tb=gen_vDAC_Fuzzing(width=width, inp=inp),
              attrs=[h.sim.Op(),
                     s.install.include(h.pdk.Corner.TYP),
                     h.sim.Include(
                            sky130.install.pdk_path /
                            "libs.ref/sky130_fd_sc_hd/spice/sky130_fd_sc_hd.spice"
                    )
                ]
            )

    return sim

sim_list = []
for i in range(-2**7,2**7):
    sim_list.append(get_sim(width=8,inp=i))

rv = vsp.sim(
    h.sim.to_proto(sim_list),
    vsp.SimOptions(
        simulator=vsp.SupportedSimulators.NGSPICE,
        fmt=vsp.ResultFormat.SIM_DATA,
        # rundir='./scratch',
    )
)

In [ ]:
import hdl21 as h
from ro import *

@h.sim.sim
class TransientSim:

    @h.module
    class Tb:

        # Params
        w_param = 8

        # Ports and Signals
        VSS = h.port()
        VDD,OUT,ctrl = 3*h.Signal()
        inp = h.Signal(width=w_param)

        # Components
        vdd  = h.PulseVoltageSource(
            delay=0,
            v1=0,
            v2=1.8,
            period=0,
            rise=10*n,
            fall=1*n,
            width=1
        )(p=VDD,n=VSS)

        signal_in = bus_signal(width=w_param, inp=32)(vout=inp, vss=VSS)
        vdac = gen_vDAC(ncells=w_param)(inp_bus=signal_in.vout, enable=VDD)

        vcro = gen_VCRO(stages=13)(VSS=VSS,
                                   VDD=VDD,
                                   CTRL=vdac.vout,
                                   OUT=OUT)
        
    # Simulation Controls
    tran = h.sim.Tran(tstop=200*n, tstep=1*p)
    inc_base = sky130.install.include(h.pdk.Corner.TYP)
    inc_cells = h.sim.Include(
            sky130.install.pdk_path /
            "libs.ref/sky130_fd_sc_hd/spice/sky130_fd_sc_hd.spice"
    )
